In [ ]:
"""
# You may need to set the BEETSDIR environment variable to point to your
# local beets configuration directory. 
import os
os.environ["BEETSDIR"] = os.path.abspath("../local/config/beets")
"""

# Preview & Import Session with `beets-flask`

We can continue an import of any existing session we started by using custom serialization logic. We accomplish this by implementing our own beet session and task logic. This allows us to save the state of our import session and resume it later. 

For example we might want to generate a preview automatically and then manually trigger imports into our beets library afterwards.

## Triggering an preview (candidate fetch)

Let's assume we have some music files `./local/inbox/album` for which we want to fetch the candidates.


In [2]:
from pathlib import Path

album_path = Path("../local/music/inbox/Rush")

For this album we can start a resumable import by creating a session state object and then calling any session with the state object. For instance triggering an preview would use the following:

In [ ]:
from beets_flask.importer.session import PreviewSession, SessionState

# Create session state
session_state = SessionState(album_path)

# Create a preview session
session = PreviewSession(session_state)

# Trigger the fetch of the candidates
await session.run_async()

[INFO] beets-flask: Running PreviewSession on state<self.state.id='bbc53b71-445c-4097-b1c0-ec43d9ea20d6'>.
[INFO] beets-flask: Completed PreviewSession on state<self.state.id='bbc53b71-445c-4097-b1c0-ec43d9ea20d6'>.


SessionState(id='bbc53b71-445c-4097-b1c0-ec43d9ea20d6', _task_states=[TaskState(id='dd3cac9f-a4ce-4146-868d-ce8049e4aa8b', task=<beets.importer.ImportTask object at 0x7f65a02fd010>, candidate_states=[CandidateState(id='bcc56028-ab04-42cf-953d-abc2677a24c9', duplicate_ids=[117], match=AlbumMatch(distance=<beets.autotag.hooks.Distance object at 0x7f65a0324690>, info={'album': 'Rush / Typhoon', 'album_id': '2dwN4ymp8jSF0gROx5LjSx', 'artist': 'Basstripper', 'artist_id': '1tSiIyp5dxfbEaS0nZGMEl', 'artists': [], 'artists_ids': [], 'tracks': [{'title': 'Rush', 'track_id': '0d02JO9X19OH6xgqebGATC', 'release_track_id': None, 'artist': 'Basstripper', 'artist_id': '1tSiIyp5dxfbEaS0nZGMEl', 'artists': [], 'artists_ids': [], 'length': 274.3, 'index': 1, 'media': None, 'medium': 1, 'medium_index': 1, 'medium_total': 2, 'artist_sort': None, 'artists_sort': [], 'disctitle': None, 'artist_credit': None, 'artists_credit': [], 'data_source': 'Spotify', 'data_url': 'https://open.spotify.com/track/0d02JO9X

The session state now contains the all the session information which is needed to reconstruct the session.

The session state object a hierachy close to the beets internal logic:
- SessionState: Reflects the state of the import session.
- TaskState: Reflects a beetstask, but they dont have such a precise real-life meaning.
- CandidateState: Reflects a beets match (i.e. a candidate the user might choose)

In [28]:
for c in session_state.task_states[0].candidate_states:
    print(f"{c.distance} | {c.album:>30} | {c.match.info.data_source}")

0.00 |                 Rush / Typhoon | Spotify
0.60 |      Punishment / Blatherskite | MusicBrainz
0.63 |                 Cascade / Move | MusicBrainz
0.63 |          Greedy / Darkest Void | MusicBrainz
0.71 |              Hazmat / Deserted | MusicBrainz
0.72 |                 RawZ / Typhoon | MusicBrainz
0.00 |                 Rush / Typhoon | asis


## Continue with an import after preview

We can use this session to continue a import later. For example if we want to choose the best candidate, we could use the AutoImportSession.

In [31]:
from beets_flask.importer.session import ImportSession

In [ ]:
# TODO